In [39]:
from google.colab import files
uploaded = files.upload()

Saving saas_billing_train.csv to saas_billing_train (3).csv


In [40]:
import pandas as pd
df = pd.read_csv('saas_billing_train (3).csv')

In [41]:
df.head()

,invoice_id,customer_id,invoice_date,due_date,payment_date,invoice_amount,tax_amount,total_amount,currency,payment_status,...,refund_amount,retries,gateway_fee,failed_reason,is_prorated,usage_units,usage_cost,rounding_diff,system_version,anomaly_type
0,1,8270,2024-10-19,2024-04-14,2024-09-15,898.57,62.00,865.120,USD,paid,...,206.79,1,8.34,NaN,0,406,426.81,0.890,v1.2.1,normal
1,2,1860,2024-10-22,2024-05-27,2024-06-24,710.02,86.13,727.713,EUR,pending,...,284.82,0,14.69,network_error,0,395,354.95,-0.967,v1.2.1,normal
2,3,6390,2024-08-20,2024-10-21,2024-01-15,754.67,180.18,930.520,EUR,failed,...,135.78,0,11.84,network_error,0,459,106.10,0.970,v1.3.0,normal
3,4,6191,2024-01-09,2024-02-10,2024-12-10,952.87,154.15,1031.775,GBP,pending,...,173.84,2,11.22,insufficient_funds,0,121,581.83,0.455,v1.4.2,normal
4,5,6734,2024-05-08,2024-12-31,2024-05-04,670.21,122.86,749.286,EUR,failed,...,52.00,2,13.14,NaN,0,279,408.33,0.326,v1.2.1,normal


In [42]:
df["invoices_per_customer"] = df.groupby("customer_id")["invoice_id"].transform("count")
df["avg_invoice_amount_customer"] = df.groupby("customer_id")["total_amount"].transform("mean")

# --- STEP 2: Drop unique identifiers ---
df = df.drop(columns=["invoice_id", "customer_id"], errors="ignore")

print("Remaining columns:", df.columns.tolist())
print(df.head())

Remaining columns: ['invoice_date', 'due_date', 'payment_date', 'invoice_amount', 'tax_amount', 'total_amount', 'currency', 'payment_status', 'payment_method', 'subscription_plan', 'billing_cycle', 'country', 'discount_applied', 'refund_amount', 'retries', 'gateway_fee', 'failed_reason', 'is_prorated', 'usage_units', 'usage_cost', 'rounding_diff', 'system_version', 'anomaly_type', 'invoices_per_customer', 'avg_invoice_amount_customer']
  invoice_date    due_date payment_date  invoice_amount  tax_amount  \
0   2024-10-19  2024-04-14   2024-09-15          898.57       62.00   
1   2024-10-22  2024-05-27   2024-06-24          710.02       86.13   
2   2024-08-20  2024-10-21   2024-01-15          754.67      180.18   
3   2024-01-09  2024-02-10   2024-12-10          952.87      154.15   
4   2024-05-08  2024-12-31   2024-05-04          670.21      122.86   

   total_amount currency payment_status payment_method subscription_plan  ...  \
0       865.120      USD           paid           ca

In [43]:
import pandas as pd
import numpy as np
df["invoice_date"] = pd.to_datetime(df["invoice_date"], errors="coerce")

# 1. Invoice age
df["invoice_age_days"] = (pd.to_datetime("today") - df["invoice_date"]).dt.days

# 2. Standard date parts
df["month"] = df["invoice_date"].dt.month
df["quarter"] = df["invoice_date"].dt.quarter
df["day_of_week"] = df["invoice_date"].dt.dayofweek
df["week_of_year"] = df["invoice_date"].dt.isocalendar().week.astype(int)

# 3. Cyclical signals
df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

df["dow_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
df["dow_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)

# 4. Drop raw date


print(df.head())
print(df.shape)

  invoice_date    due_date payment_date  invoice_amount  tax_amount  \
0   2024-10-19  2024-04-14   2024-09-15          898.57       62.00   
1   2024-10-22  2024-05-27   2024-06-24          710.02       86.13   
2   2024-08-20  2024-10-21   2024-01-15          754.67      180.18   
3   2024-01-09  2024-02-10   2024-12-10          952.87      154.15   
4   2024-05-08  2024-12-31   2024-05-04          670.21      122.86   

   total_amount currency payment_status payment_method subscription_plan  ...  \
0       865.120      USD           paid           card        enterprise  ...   
1       727.713      EUR        pending           card               pro  ...   
2       930.520      EUR         failed         paypal               pro  ...   
3      1031.775      GBP        pending         paypal             basic  ...   
4       749.286      EUR         failed  bank_transfer        enterprise  ...   

  avg_invoice_amount_customer invoice_age_days  month  quarter  day_of_week  \
0      

In [44]:
# ============================
# FEATURE ENGINEERING: Due Date & Payment Date
# ============================

import pandas as pd
import numpy as np

# Ensure datetime conversion
df["due_date"] = pd.to_datetime(df["due_date"], errors="coerce")
df["payment_date"] = pd.to_datetime(df["payment_date"], errors="coerce")

# ============================
# 1. Date Differences (Strong Predictors)
# ============================

# How long after invoice until due?
df["billing_cycle_days"] = (df["due_date"] - pd.to_datetime(df["invoice_date"])).dt.days

# How many days before/after due date was payment made?
df["days_late"] = (df["payment_date"] - df["due_date"]).dt.days

# Time between invoice and payment
df["days_to_payment"] = (df["payment_date"] - pd.to_datetime(df["invoice_date"])).dt.days

# Time remaining until due date (from invoice issue)
df["days_until_due"] = (df["due_date"] - pd.to_datetime(df["invoice_date"])).dt.days

# ============================
# 2. Standard Date Parts: Due Date
# ============================

df["due_month"] = df["due_date"].dt.month
df["due_dow"] = df["due_date"].dt.dayofweek
df["due_week"] = df["due_date"].dt.isocalendar().week.astype(int)

# ============================
# 3. Cyclical Encoding: Due Date
# ============================

df["due_month_sin"] = np.sin(2 * np.pi * df["due_month"] / 12)
df["due_month_cos"] = np.cos(2 * np.pi * df["due_month"] / 12)

df["due_dow_sin"] = np.sin(2 * np.pi * df["due_dow"] / 7)
df["due_dow_cos"] = np.cos(2 * np.pi * df["due_dow"] / 7)

# ============================
# 4. Standard Date Parts: Payment Date
# ============================

df["pay_month"] = df["payment_date"].dt.month
df["pay_dow"] = df["payment_date"].dt.dayofweek
df["pay_week"] = df["payment_date"].dt.isocalendar().week.astype(int)

# ============================
# 5. Cyclical Encoding: Payment Date
# ============================

df["pay_month_sin"] = np.sin(2 * np.pi * df["pay_month"] / 12)
df["pay_month_cos"] = np.cos(2 * np.pi * df["pay_month"] / 12)

df["pay_dow_sin"] = np.sin(2 * np.pi * df["pay_dow"] / 7)
df["pay_dow_cos"] = np.cos(2 * np.pi * df["pay_dow"] / 7)

# ============================
# 6. Drop raw dates
# Keep only engineered signals
# ============================
df = df.drop(columns=["invoice_date"])
df = df.drop(columns=["due_date", "payment_date"])

print(df.head())
print(df.shape)


   invoice_amount  tax_amount  total_amount currency payment_status  \
0          898.57       62.00       865.120      USD           paid   
1          710.02       86.13       727.713      EUR        pending   
2          754.67      180.18       930.520      EUR         failed   
3          952.87      154.15      1031.775      GBP        pending   
4          670.21      122.86       749.286      EUR         failed   

  payment_method subscription_plan billing_cycle country  discount_applied  \
0           card        enterprise       monthly      US             96.34   
1           card               pro        yearly      US             67.47   
2         paypal               pro     quarterly      UK              5.30   
3         paypal             basic     quarterly      IN             75.70   
4  bank_transfer        enterprise     quarterly      UK             44.11   

   ...  due_month_cos  due_dow_sin  due_dow_cos pay_month  pay_dow  pay_week  \
0  ...      -0.500000   

In [45]:
# 4. AMOUNT FEATURES (invoice, tax, total)
# ============================================================

# Keep original numeric columns
# Add engineered versions

# Effective tax rate
df["tax_rate"] = df["tax_amount"] / df["invoice_amount"]

# Relationship check (detect discount/adjustments)
df["amount_diff"] = df["total_amount"] - (df["invoice_amount"] + df["tax_amount"])
df["discount_flag"] = (df["amount_diff"] < 0).astype(int)

# Log scale versions
df["invoice_amount_log"] = np.log1p(df["invoice_amount"])
df["tax_amount_log"] = np.log1p(df["tax_amount"])
df["total_amount_log"] = np.log1p(df["total_amount"])

# Ratio features
df["tax_to_total_ratio"] = df["tax_amount"] / df["total_amount"]
df["amount_to_total_ratio"] = df["invoice_amount"] / df["total_amount"]

# ============================================================
# OPTIONAL: DROP RAW DATES
# ============================================================


print(df.head())
print(df.shape)

   invoice_amount  tax_amount  total_amount currency payment_status  \
0          898.57       62.00       865.120      USD           paid   
1          710.02       86.13       727.713      EUR        pending   
2          754.67      180.18       930.520      EUR         failed   
3          952.87      154.15      1031.775      GBP        pending   
4          670.21      122.86       749.286      EUR         failed   

  payment_method subscription_plan billing_cycle country  discount_applied  \
0           card        enterprise       monthly      US             96.34   
1           card               pro        yearly      US             67.47   
2         paypal               pro     quarterly      UK              5.30   
3         paypal             basic     quarterly      IN             75.70   
4  bank_transfer        enterprise     quarterly      UK             44.11   

   ...  pay_dow_sin  pay_dow_cos  tax_rate amount_diff  discount_flag  \
0  ...    -0.781831     0.62349

In [46]:
import pandas as pd

cat_cols = [
    "currency",
    "payment_status",
    "payment_method",
    "subscription_plan",
    "billing_cycle",

]

# One-hot encode (drop_first=False because anomalies need full info)
df = pd.get_dummies(df, columns=cat_cols, drop_first=False)

print(df.shape)
print(df.head())


(75000, 68)
   invoice_amount  tax_amount  total_amount country  discount_applied  \
0          898.57       62.00       865.120      US             96.34   
1          710.02       86.13       727.713      US             67.47   
2          754.67      180.18       930.520      UK              5.30   
3          952.87      154.15      1031.775      IN             75.70   
4          670.21      122.86       749.286      UK             44.11   

   refund_amount  retries  gateway_fee       failed_reason  is_prorated  ...  \
0         206.79        1         8.34                 NaN            0  ...   
1         284.82        0        14.69       network_error            0  ...   
2         135.78        0        11.84       network_error            0  ...   
3         173.84        2        11.22  insufficient_funds            0  ...   
4          52.00        2        13.14                 NaN            0  ...   

   payment_status_pending  payment_method_bank_transfer  payment_met

In [47]:
df["country_freq"] = df["country"].map(df["country"].value_counts() / len(df))

df = df.drop(columns=["country"])


In [48]:
# FAILED_REASON Feature Engineering ----------------------------

# 1. Binary flag
df["has_failed_reason"] = df["failed_reason"].notna().astype(int)

# 2. Replace NaN with "none"
df["failed_reason"] = df["failed_reason"].fillna("none")

# 3. Frequency encoding
freq_map = df["failed_reason"].value_counts(normalize=True)
df["failed_reason_freq"] = df["failed_reason"].map(freq_map)

# 4. Drop raw column
df = df.drop(columns=["failed_reason"])


In [49]:
# Convert all True/False columns to 0/1
bool_cols = df.select_dtypes(include=["bool"]).columns

df[bool_cols] = df[bool_cols].astype(int)

print(f"Converted {len(bool_cols)} boolean columns to 0/1.")


Converted 16 boolean columns to 0/1.


In [50]:
df.columns

Index(['invoice_amount', 'tax_amount', 'total_amount', 'discount_applied',
       'refund_amount', 'retries', 'gateway_fee', 'is_prorated', 'usage_units',
       'usage_cost', 'rounding_diff', 'system_version', 'anomaly_type',
       'invoices_per_customer', 'avg_invoice_amount_customer',
       'invoice_age_days', 'month', 'quarter', 'day_of_week', 'week_of_year',
       'month_sin', 'month_cos', 'dow_sin', 'dow_cos', 'billing_cycle_days',
       'days_late', 'days_to_payment', 'days_until_due', 'due_month',
       'due_dow', 'due_week', 'due_month_sin', 'due_month_cos', 'due_dow_sin',
       'due_dow_cos', 'pay_month', 'pay_dow', 'pay_week', 'pay_month_sin',
       'pay_month_cos', 'pay_dow_sin', 'pay_dow_cos', 'tax_rate',
       'amount_diff', 'discount_flag', 'invoice_amount_log', 'tax_amount_log',
       'total_amount_log', 'tax_to_total_ratio', 'amount_to_total_ratio',
       'currency_EUR', 'currency_GBP', 'currency_INR', 'currency_USD',
       'payment_status_failed', 'payment_

In [51]:
df.head()

,invoice_amount,tax_amount,total_amount,discount_applied,refund_amount,retries,gateway_fee,is_prorated,usage_units,usage_cost,...,payment_method_paypal,subscription_plan_basic,subscription_plan_enterprise,subscription_plan_pro,billing_cycle_monthly,billing_cycle_quarterly,billing_cycle_yearly,country_freq,has_failed_reason,failed_reason_freq
0,898.57,62.00,865.120,96.34,206.79,1,8.34,0,406,426.81,...,0,0,1,0,1,0,0,0.199507,0,0.250067
1,710.02,86.13,727.713,67.47,284.82,0,14.69,0,395,354.95,...,0,0,0,1,0,0,1,0.199507,1,0.250907
2,754.67,180.18,930.520,5.30,135.78,0,11.84,0,459,106.10,...,1,0,0,1,0,1,0,0.200227,1,0.250907
3,952.87,154.15,1031.775,75.70,173.84,2,11.22,0,121,581.83,...,1,1,0,0,0,1,0,0.199880,1,0.250373
4,670.21,122.86,749.286,44.11,52.00,2,13.14,0,279,408.33,...,0,0,1,0,0,1,0,0.200227,0,0.250067


In [52]:
cols_to_drop = [
    # "anomaly_type",          # target-like label (not allowed for unsupervised) - KEEP FOR EVALUATION
    "discount_applied",      # redundant because we created discount_flag
    "failed_reason_freq",    # weak signal, noisy frequency encoding

    # Raw date parts removed because we kept cyclic versions instead
    "month", "quarter", "day_of_week", "week_of_year",
    "due_month", "due_dow", "due_week",
    "pay_month", "pay_dow", "pay_week",
]

df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

print("Dropped:", cols_to_drop)
print("Remaining feature count:", df.shape[1])

Dropped: ['discount_applied', 'failed_reason_freq', 'month', 'quarter', 'day_of_week', 'week_of_year', 'due_month', 'due_dow', 'due_week', 'pay_month', 'pay_dow', 'pay_week']
Remaining feature count: 57


In [53]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(train_df.shape, test_df.shape)


(60000, 57) (15000, 57)


In [54]:
!pip install keras

In [55]:
# ============================================
# UNSUPERVISED ANOMALY MODEL EVALUATION
# Using your existing train_df and test_df
# ============================================

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import RobustScaler
from tensorflow import keras
from tensorflow.keras import layers

# -----------------------------
# CONFIG
# -----------------------------
TOP_K_PERCENT = 0.10  # precision@10%
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# -----------------------------
# UTIL — Precision@K
# -----------------------------
def precision_at_k(y_true_bool, scores, k=0.10):
    n = len(scores)
    top_n = max(1, int(n * k))
    top_idx = np.argsort(scores)[::-1][:top_n]
    return y_true_bool.iloc[top_idx].mean()


# -----------------------------
# 1) PREPARE TRAIN & TEST
# -----------------------------
def prepare(df):
    """Remove label + non-numeric, fill NA."""
    df = df.copy()

    # ground truth (only for evaluation)
    y_true = df["anomaly_type"].apply(lambda x: x != "normal")

    # remove label
    df = df.drop(columns=["anomaly_type"], errors="ignore")

    # numeric only
    df = df.select_dtypes(include=[np.number])

    # fill NA
    df = df.fillna(0)

    return df, y_true


train_X, train_y_true = prepare(train_df)
test_X,  test_y_true  = prepare(test_df)

print("Train shape:", train_X.shape)
print("Test shape :", test_X.shape)


# -----------------------------
# 2) SCALING
# -----------------------------
scaler = RobustScaler()
train_scaled = scaler.fit_transform(train_X)
test_scaled  = scaler.transform(test_X)


# -----------------------------
# AUTOENCODER MODEL BUILDER
# -----------------------------
def build_autoencoder(input_dim):
    inp = layers.Input(shape=(input_dim,))
    x = layers.Dense(64, activation="relu")(inp)
    x = layers.Dense(32, activation="relu")(x)
    bottleneck = layers.Dense(16, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(bottleneck)
    x = layers.Dense(64, activation="relu")(x)
    out = layers.Dense(input_dim, activation="linear")(x)
    model = keras.Model(inp, out)
    model.compile(optimizer="adam", loss="mse")
    return model


# -----------------------------
# STORE RESULTS
# -----------------------------
results = []

# -----------------------------
# 3) Isolation Forest
# -----------------------------
print("\nTraining Isolation Forest...")
iso = IsolationForest(
    n_estimators=200,
    contamination='auto',
    random_state=RANDOM_STATE
)
iso.fit(train_scaled)

# higher score = more anomaly → invert sign
iso_scores = -iso.decision_function(test_scaled)

iso_precision = precision_at_k(test_y_true, iso_scores, k=TOP_K_PERCENT)
results.append(("IsolationForest", iso_precision))
print("Isolation Forest precision@10%:", iso_precision)


# -----------------------------
# 4) Local Outlier Factor
# -----------------------------
print("\nTraining LOF...")
lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination='auto',
    novelty=False
)

# Fit on train+test combined (LOF requirement)
combined = np.vstack([train_scaled, test_scaled])
lof.fit(combined)

# LOF scores only available after fit_predict, so redo on combined
lof_pred = lof.fit_predict(combined)
lof_scores_all = -(lof.negative_outlier_factor_)

# Extract test scores only
lof_scores = lof_scores_all[len(train_scaled):]

lof_precision = precision_at_k(test_y_true, lof_scores, k=TOP_K_PERCENT)
results.append(("LOF", lof_precision))
print("LOF precision@10%:", lof_precision)


# -----------------------------
# 5) One-Class SVM
# -----------------------------
print("\nTraining One-Class SVM (slow)...")
oc = OneClassSVM(kernel="rbf", gamma="scale", nu=0.01)
oc.fit(train_scaled)

oc_scores = -oc.decision_function(test_scaled).ravel()

oc_precision = precision_at_k(test_y_true, oc_scores, k=TOP_K_PERCENT)
results.append(("OneClassSVM", oc_precision))
print("One-Class SVM precision@10%:", oc_precision)


# -----------------------------
# 6) Autoencoder
# -----------------------------
print("\nTraining Autoencoder...")
input_dim = train_scaled.shape[1]
ae = build_autoencoder(input_dim)

ae.fit(
    train_scaled, train_scaled,
    epochs=10, batch_size=256, verbose=0
)

# Reconstruction error
recon = ae.predict(test_scaled, verbose=0)
ae_scores = np.mean(np.square(test_scaled - recon), axis=1)

ae_precision = precision_at_k(test_y_true, ae_scores, k=TOP_K_PERCENT)
results.append(("Autoencoder", ae_precision))
print("Autoencoder precision@10%:", ae_precision)


# -----------------------------
# 7) FINAL RESULTS TABLE
# -----------------------------
print("\n==========================")
print(" FINAL MODEL PRECISIONS ")
print("==========================")
for name, p in results:
    print(f"{name:20s}  precision@10% = {p:.4f}")

best_model = max(results, key=lambda x: x[1])
print("\nBEST MODEL:", best_model[0], " | precision@10% =", best_model[1])


Train shape: (60000, 55)
Test shape : (15000, 55)

Training Isolation Forest...
Isolation Forest precision@10%: 0.23066666666666666

Training LOF...
LOF precision@10%: 0.19

Training One-Class SVM (slow)...
One-Class SVM precision@10%: 0.24266666666666667

Training Autoencoder...
Autoencoder precision@10%: 0.32066666666666666

 FINAL MODEL PRECISIONS 
IsolationForest       precision@10% = 0.2307
LOF                   precision@10% = 0.1900
OneClassSVM           precision@10% = 0.2427
Autoencoder           precision@10% = 0.3207

BEST MODEL: Autoencoder  | precision@10% = 0.32066666666666666
